# Propose BERT siamese with triplet

In [1]:
# import tensorflow as tf
import keras
# from tensorflow.python import keras
import os

Using TensorFlow backend.


In [2]:
# import tensorflow as tf
# import tensorflow.keras.backend as K_tf

# sess = K_tf.get_session()
# uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
# init_op = tf.variables_initializer(
#     [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
# )
# sess.run(init_op)

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 1000
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'netbeans'
METHOD = 'propose_bert_triplet_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [10]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [11]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


216715

#### Dicionário de títulos e descrições

In [12]:
# !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip -o uncased_L-12_H-768_A-12.zip

In [13]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [14]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [15]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [16]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 16.6 s, sys: 2.14 s, total: 18.8 s
Wall time: 18.8 s


#### Hashing bugs by buckets

In [17]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [18]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 2min 34s, sys: 28.3 ms, total: 2min 34s
Wall time: 2min 34s


#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [20]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '0\n',
 'bug_status': '1\n',
 'component': '470\n',
 'creation_ts': '2005-04-21 01:37:00 +0000',
 'delta_ts': '2008-12-22 22:47:29 +0000',
 'description': '[CLS] copy of issue info : xp with default color scheme arguably not a shows ##top ##per but really reflects on the product quality . new car with a dent kind of thing . see attachment . [SEP]',
 'description_bert': '[CLS] copy of issue info : xp with default color scheme arguably not a shows ##top ##per but really reflects on the product quality . new car with a dent kind of thing . see attachment . [SEP]',
 'description_word': array([  101,  6100,  1997,  3277, 18558,  1024, 26726,  2007, 12398,
         3609,  5679, 15835,  2025,  1037,  3065, 14399,  4842,  2021,
         2428, 11138]),
 'description_word_bert': [101,
  6100,
  1997,
  3277,
  18558,
  1024,
  26726,
  2007,
  12398,
  3609,
  5679,
  15835,
  2025,
  1037,
  3065,
  14399,
  4842,
  2021,
  2428,
  11138,
  2006,
  1996,
  4031,
  3737,
  1012,

### Generating the batch test

In [21]:
"Train ", len(baseline.dup_sets_train)

('Train ', 30600)

In [22]:
bug_idx = bug_train_ids[0]
vector = baseline.bug_set[bug_idx]['textual_word']
annoy_train = AnnoyIndex(vector.shape[0])
for bug_id in bug_train_ids:
    annoy_train.add_item(bug_id, baseline.bug_set[bug_id]['textual_word'])
annoy_train.build(10) # 10 trees
"Indexed all train"

'Indexed all train'

### Train ids

In [23]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [24]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator_bert(None, 
                                                                                                                  baseline.train_data, 
                                                                                                                  baseline.dup_sets_train,
                                                                                                                  bug_train_ids,
                                                                                                                  batch_size_test, 1, 
                                                                                                                  issues_by_buckets)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 54.8 ms, sys: 0 ns, total: 54.8 ms
Wall time: 54.3 ms


In [25]:
valid_input_sample['title']['token'].shape, valid_input_sample['description']['token'].shape, \
    valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 544), (128,))

### Validar entrada

In [26]:
# %%time 

# baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5, batch_iterator, issues_by_buckets)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [27]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=False,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
#     outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [30]:
from keras import backend as K

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)

def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

### Propose

In [31]:
from keras.layers import concatenate, Add, Lambda, Average, Maximum, Subtract, Average, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
    
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [32]:
from keras.layers import Average
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])

    # Loss function only works with a single output
    output = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances-bug',
        output_shape=(2, 1)
    )([positive_d, negative_d])
    
    #output_avg_master = Average()([output_master, output_master_pos, output_master_neg])
    #output = Average()([output_bug, output_avg_master])
    #loss = MarginLoss()(output)

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_margin_loss, 
                                 metrics=[pos_distance, neg_distance])

    return similarity_model

In [ ]:
%%time
import os

print("Batch size ", batch_size)

# Inspired on https://'pastebin.com/TaGFdcBA
keras.backend.clear_session()

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
# title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
#desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')


NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_sim = experiment.batch_iterator_bert(encoded_anchor, baseline.train_data, baseline.dup_sets_train, bug_train_ids, 
                                       batch_size, 1, issues_by_buckets)
    
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], train_input_sample['description']['token'], train_input_sample['description']['segment'], train_input_sample['info'],
                   train_input_pos['title']['token'], train_input_pos['title']['segment'], train_input_pos['description']['token'], train_input_pos['description']['segment'], train_input_pos['info'], 
                   train_input_neg['title']['token'], train_input_neg['title']['segment'], train_input_neg['description']['token'], train_input_neg['description']['segment'], train_input_neg['info']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, recall@25: {:.2f}".format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1], h[2], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}".format(epoch+1,
                                                                                                         h[0],
                                                                                                         h[1],
                                                                                                         h[2]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}s, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

Batch size  64
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 544)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
______________________________________________________________________________________________

Epoch: 19 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 20 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 21 Loss: 1.01, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 22 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.99
Epoch: 23 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.99
Epoch: 24 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.99
Epoch: 25 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 26 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 27 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 28 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 29 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 30 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 31 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 32 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 33 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 34 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 35 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 36 Loss: 1.00, pos_cosin

Epoch: 162 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 163 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 164 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 165 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 166 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 167 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 168 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 169 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 170 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 171 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 172 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 173 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 174 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 175 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 176 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 177 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 178 Loss: 1.00, pos_cosine: 1.00, neg_cosine: 1.00
Epoch: 179 Los

Epoch: 304 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 305 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.98
Epoch: 306 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 307 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 308 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 309 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 310 Loss: 1.00, pos_cosine: 0.97, neg_cosine: 0.97
Epoch: 311 Loss: 1.00, pos_cosine: 0.97, neg_cosine: 0.98
Epoch: 312 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 313 Loss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 314 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 315 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 316 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 317 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 318 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 319 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 320 Loss: 1.00, pos_cosine: 0.99, neg_cosine: 0.99
Epoch: 321 Los

Epoch: 446 Loss: 0.96, pos_cosine: 0.78, neg_cosine: 0.75
Epoch: 447 Loss: 0.95, pos_cosine: 0.80, neg_cosine: 0.75
Epoch: 448 Loss: 0.94, pos_cosine: 0.82, neg_cosine: 0.76
Epoch: 449 Loss: 0.97, pos_cosine: 0.79, neg_cosine: 0.75
Epoch: 450 Loss: 0.97, pos_cosine: 0.79, neg_cosine: 0.76
Epoch: 451 Loss: 0.97, pos_cosine: 0.79, neg_cosine: 0.75
Epoch: 452 Loss: 0.98, pos_cosine: 0.79, neg_cosine: 0.77
Epoch: 453 Loss: 0.96, pos_cosine: 0.80, neg_cosine: 0.76
Epoch: 454 Loss: 0.98, pos_cosine: 0.76, neg_cosine: 0.74
Epoch: 455 Loss: 0.97, pos_cosine: 0.79, neg_cosine: 0.75
Epoch: 456 Loss: 0.95, pos_cosine: 0.80, neg_cosine: 0.75
Epoch: 457 Loss: 0.95, pos_cosine: 0.79, neg_cosine: 0.74
Epoch: 458 Loss: 0.96, pos_cosine: 0.78, neg_cosine: 0.74
Epoch: 459 Loss: 0.97, pos_cosine: 0.78, neg_cosine: 0.75
Epoch: 460 Loss: 0.97, pos_cosine: 0.78, neg_cosine: 0.75
Epoch: 461 Loss: 0.97, pos_cosine: 0.79, neg_cosine: 0.76
Epoch: 462 Loss: 0.96, pos_cosine: 0.78, neg_cosine: 0.74
Epoch: 463 Los

Epoch: 588 Loss: 0.92, pos_cosine: 0.81, neg_cosine: 0.74
Epoch: 589 Loss: 0.97, pos_cosine: 0.78, neg_cosine: 0.75
Epoch: 590 Loss: 0.95, pos_cosine: 0.77, neg_cosine: 0.72
Epoch: 591 Loss: 0.93, pos_cosine: 0.78, neg_cosine: 0.71
Epoch: 592 Loss: 0.95, pos_cosine: 0.77, neg_cosine: 0.72
Epoch: 593 Loss: 0.99, pos_cosine: 0.76, neg_cosine: 0.75
Epoch: 594 Loss: 0.95, pos_cosine: 0.79, neg_cosine: 0.75
Epoch: 595 Loss: 0.96, pos_cosine: 0.76, neg_cosine: 0.71
Epoch: 596 Loss: 0.94, pos_cosine: 0.78, neg_cosine: 0.73
Epoch: 597 Loss: 0.94, pos_cosine: 0.79, neg_cosine: 0.73
Epoch: 598 Loss: 0.93, pos_cosine: 0.81, neg_cosine: 0.74
Epoch: 599 Loss: 0.97, pos_cosine: 0.76, neg_cosine: 0.74
Epoch: 600 Loss: 0.93, pos_cosine: 0.80, neg_cosine: 0.73
Epoch: 601 Loss: 0.97, pos_cosine: 0.77, neg_cosine: 0.74
Epoch: 602 Loss: 0.95, pos_cosine: 0.78, neg_cosine: 0.72
Epoch: 603 Loss: 0.94, pos_cosine: 0.77, neg_cosine: 0.71
Epoch: 604 Loss: 0.93, pos_cosine: 0.80, neg_cosine: 0.73
Epoch: 605 Los

In [ ]:
recall

In [ ]:
_[:20]

In [ ]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [ ]:
print("Total of queries:", len(retrieval.test))

#### Getting the model trained

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [ ]:
model.summary()

In [ ]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='bert')

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report